In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# !pip install torch==1.13.1+cu116 torchvision==0.14.1+cu116 --extra-index-url https://download.pytorch.org/whl/cu116
# !pip install matplotlib numpy pandas tqdm nltk

# for separating ingredients vs non-ingredients
# NOTE: if using Windows to run this, need to download GNU Wget
# !wget -c https://raw.githubusercontent.com/williamLyh/RecipeWithPlans/main/ingredient_set.json -O ingredient_set.json

In [3]:
import os
import math
import re
import string
import numpy as np
import pandas as pd
import random
import json
from torch.optim.lr_scheduler import CosineAnnealingLR, StepLR, MultiStepLR, CosineAnnealingWarmRestarts
import nltk
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu
from nltk.translate import meteor

from data import *
from encoder_decoder import *
from train import *
from eval import *
from utils import *
from neuro_dec import *

# required for bleu
nltk.download("wordnet")
nltk.download("punkt")

/home/student/anaconda3/envs/nlp/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package wordnet to /home/student/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/student/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

---

In [4]:
SEED = 31989101
HIDDEN_SIZE = 256
MAX_INGR_LEN = 150 # fixed from assignment
MAX_RECIPE_LEN = 600
DROPOUT = 0.1
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## ensuring reproducibility
def reset_rng():
    torch.manual_seed(SEED)
    np.random.seed(SEED)
    random.seed(SEED)

reset_rng()

# to easily read ingredients and instructions
pd.set_option('display.max_colwidth', 2000)

print(f"Using device: {DEVICE}")

Using device: cuda


In [5]:
data_root = "./Cooking_Dataset"
add_intermediate_tag=False

train_df_orig = pd.read_csv(os.path.join(data_root, "train.csv"), usecols=['Ingredients', 'Recipe'])
dev_df_orig = pd.read_csv(os.path.join(data_root, "dev.csv"), usecols=['Ingredients', 'Recipe'])
test_df_orig = pd.read_csv(os.path.join(data_root, "test.csv"), usecols=['Ingredients', 'Recipe'])

In [6]:
train_df = preprocess_data(train_df_orig, max_ingr_len=MAX_INGR_LEN, max_recipe_len=MAX_RECIPE_LEN, add_intermediate_tag=add_intermediate_tag)

Number of data samples before preprocessing: 101340
Number of data samples after preprocessing: 100637 (99.306%)


In [7]:
dev_df = preprocess_data(dev_df_orig, max_ingr_len=MAX_INGR_LEN, max_recipe_len=MAX_RECIPE_LEN, add_intermediate_tag=add_intermediate_tag)

Number of data samples before preprocessing: 797
Number of data samples after preprocessing: 793 (99.498%)


In [8]:
test_df = preprocess_data(test_df_orig, max_ingr_len=MAX_INGR_LEN, max_recipe_len=MAX_RECIPE_LEN, add_intermediate_tag=add_intermediate_tag)

Number of data samples before preprocessing: 778
Number of data samples after preprocessing: 774 (99.486%)


In [9]:
vocab = Vocabulary(add_intermediate_tag=add_intermediate_tag)
vocab.populate(train_df)
vocab.n_unique_words

  0%|          | 0/100637 [00:00<?, ?it/s]

100%|██████████| 100637/100637 [00:06<00:00, 15730.54it/s]


44315

In [10]:
train_ds = RecipeDataset(train_df, vocab)
# subset_train_ds = RecipeDataset(train_df[:250], vocab) # ! REMOVE LATER
dev_ds_val_loss = RecipeDataset(dev_df, vocab, train=True) # used for getting validation loss
dev_ds_val_met = RecipeDataset(dev_df, vocab, train=False) # used for getting validation BLEU, and other metrics
test_ds = RecipeDataset(test_df, vocab, train=False)

## Encoder-Decoder (Base)

In [11]:
embedding_size=300
encoder = EncoderRNN(vocab.n_unique_words, embedding_size=embedding_size, hidden_size=HIDDEN_SIZE, padding_value=vocab.word2index(PAD_WORD)).to(DEVICE)
# in the training script, decoder is always fed a non-end token and thus never needs to generate padding
# also it should never generate "<UNKNOWN>"
decoder = DecoderRNN(embedding_size=embedding_size,hidden_size=HIDDEN_SIZE, output_size=vocab.n_unique_words-1).to(DEVICE)

In [12]:
initial_lr=1e-3
min_lr = 1e-5
n_epochs = 30
batch_size=128
encoder_optimizer = optim.Adam(encoder.parameters(), lr=initial_lr)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=initial_lr)
enc_scheduler = CosineAnnealingLR(encoder_optimizer, T_max=n_epochs, eta_min=min_lr)
dec_scheduler = CosineAnnealingLR(decoder_optimizer, T_max=n_epochs, eta_min=min_lr)
# enc_scheduler = MultiStepLR(encoder_optimizer, milestones=[15], gamma=0.1)
# dec_scheduler = MultiStepLR(decoder_optimizer, milestones=[15], gamma=0.1)
identifier="adam_without_intermediate_tags_with_val_wd0_lr1e-3"
epoch_losses, val_epoch_losses, log = train(encoder, decoder, encoder_optimizer, decoder_optimizer, train_ds, 
                     n_epochs=n_epochs, vocab=vocab, decoder_mode="basic", batch_size=batch_size, 
                     enc_lr_scheduler=enc_scheduler, dec_lr_scheduler=dec_scheduler, 
                     dev_ds_val_loss = dev_ds_val_loss, dev_ds_val_met=dev_ds_val_met, identifier=identifier,
                     verbose_iter_interval=50)

save_log(identifier, log, encoder_optimizer, decoder_optimizer, enc_scheduler, dec_scheduler)

Starting epoch 1/30, enc lr scheduler: [0.001], dec lr scheduler: [0.001]
(Epoch 0, iter 50/787) Average loss so far: 7.144
(Epoch 0, iter 100/787) Average loss so far: 5.967
(Epoch 0, iter 150/787) Average loss so far: 5.730
(Epoch 0, iter 200/787) Average loss so far: 5.470
(Epoch 0, iter 250/787) Average loss so far: 5.212
(Epoch 0, iter 300/787) Average loss so far: 5.019
(Epoch 0, iter 350/787) Average loss so far: 4.865
(Epoch 0, iter 400/787) Average loss so far: 4.737
(Epoch 0, iter 450/787) Average loss so far: 4.632
(Epoch 0, iter 500/787) Average loss so far: 4.540
(Epoch 0, iter 550/787) Average loss so far: 4.449
(Epoch 0, iter 600/787) Average loss so far: 4.382
(Epoch 0, iter 650/787) Average loss so far: 4.317
(Epoch 0, iter 700/787) Average loss so far: 4.246
(Epoch 0, iter 750/787) Average loss so far: 4.207
Average epoch loss: 4.956
This epoch took 5.9502798040707905 mins. Time remaining: 2.0 hrs 52.0 mins.


7it [00:00,  8.22it/s]


validation loss: 4.188371658325195


100%|██████████| 793/793 [00:09<00:00, 87.30it/s] 


BLEU score: 0.0040273883646875595, METEOR score: 0.09091902403865486
Starting epoch 2/30, enc lr scheduler: [0.0009972883382072953], dec lr scheduler: [0.0009972883382072953]
(Epoch 1, iter 50/787) Average loss so far: 4.108
(Epoch 1, iter 100/787) Average loss so far: 4.071
(Epoch 1, iter 150/787) Average loss so far: 4.044
(Epoch 1, iter 200/787) Average loss so far: 4.004
(Epoch 1, iter 250/787) Average loss so far: 3.953
(Epoch 1, iter 300/787) Average loss so far: 3.925
(Epoch 1, iter 350/787) Average loss so far: 3.904
(Epoch 1, iter 400/787) Average loss so far: 3.880
(Epoch 1, iter 450/787) Average loss so far: 3.846
(Epoch 1, iter 500/787) Average loss so far: 3.816
(Epoch 1, iter 550/787) Average loss so far: 3.792
(Epoch 1, iter 600/787) Average loss so far: 3.775
(Epoch 1, iter 650/787) Average loss so far: 3.753
(Epoch 1, iter 700/787) Average loss so far: 3.747
(Epoch 1, iter 750/787) Average loss so far: 3.701
Average epoch loss: 3.878
This epoch took 5.9652024269104 min

...

BLEU score: 0.04576639965453813, METEOR score: 0.19814575935905007
Starting epoch 30/30, enc lr scheduler: [1.2711661792704668e-05], dec lr scheduler: [1.2711661792704668e-05]
(Epoch 29, iter 50/787) Average loss so far: 2.674
(Epoch 29, iter 100/787) Average loss so far: 2.674
(Epoch 29, iter 150/787) Average loss so far: 2.675
(Epoch 29, iter 200/787) Average loss so far: 2.647
(Epoch 29, iter 250/787) Average loss so far: 2.673
(Epoch 29, iter 300/787) Average loss so far: 2.653
(Epoch 29, iter 350/787) Average loss so far: 2.662
(Epoch 29, iter 400/787) Average loss so far: 2.661
(Epoch 29, iter 450/787) Average loss so far: 2.664
(Epoch 29, iter 500/787) Average loss so far: 2.695
(Epoch 29, iter 550/787) Average loss so far: 2.670
(Epoch 29, iter 600/787) Average loss so far: 2.660
(Epoch 29, iter 650/787) Average loss so far: 2.679
(Epoch 29, iter 700/787) Average loss so far: 2.666
(Epoch 29, iter 750/787) Average loss so far: 2.656
Average epoch loss: 2.668
This epoch took 5.9

7it [00:00,  8.36it/s]


validation loss: 3.191310610089983


100%|██████████| 793/793 [00:01<00:00, 697.13it/s]

BLEU score: 0.045667970073548995, METEOR score: 0.19664536921554326


## Encoder-Decoder (Attention)

In [11]:
reset_rng()

In [12]:
embedding_size=300
encoder_attn = EncoderRNN(vocab.n_unique_words, embedding_size=embedding_size, hidden_size=HIDDEN_SIZE, padding_value=vocab.word2index(PAD_WORD)).to(DEVICE)
# in the training script, decoder is always fed a non-end token and thus never needs to generate padding
# also it should never generate "<UNKNOWN>"
# decoder = DecoderRNN(embedding_size=embedding_size,hidden_size=HIDDEN_SIZE, output_size=vocab.n_unique_words-2).to(DEVICE)
decoder_attn = AttnDecoderRNN(embedding_size, hidden_size=HIDDEN_SIZE, output_size=vocab.n_unique_words-1, padding_val=vocab.word2index(PAD_WORD), 
                              dropout=DROPOUT).to(DEVICE)

In [13]:
initial_lr=1e-3
min_lr = 1e-5
n_epochs = 30
batch_size=128
encoder_attn_optimizer = optim.Adam(encoder_attn.parameters(), lr=initial_lr)
decoder_attn_optimizer = optim.Adam(decoder_attn.parameters(), lr=initial_lr)
# enc_attn_scheduler = CosineAnnealingLR(encoder_attn_optimizer, T_max=n_epochs, eta_min=min_lr)
# dec_attn_scheduler = CosineAnnealingLR(decoder_attn_optimizer, T_max=n_epochs, eta_min=min_lr)
enc_attn_scheduler = CosineAnnealingLR(encoder_attn_optimizer, T_max=n_epochs, eta_min=min_lr)
dec_attn_scheduler = CosineAnnealingLR(decoder_attn_optimizer, T_max=n_epochs, eta_min=min_lr)
identifier="attn_adam_without_intermediate_tags_wd0_lr1e-3"

attn_epoch_losses, attn_val_epoch_losses, attn_log = train(
    encoder_attn, decoder_attn, encoder_attn_optimizer, decoder_attn_optimizer, train_ds, 
    n_epochs=n_epochs, vocab=vocab, decoder_mode="attention", batch_size=batch_size, 
    enc_lr_scheduler=enc_attn_scheduler, dec_lr_scheduler=dec_attn_scheduler, 
    dev_ds_val_loss = dev_ds_val_loss, dev_ds_val_met=dev_ds_val_met, identifier=identifier,
    verbose_iter_interval=50)

save_log(identifier, attn_log, encoder_attn_optimizer, decoder_attn_optimizer, 
         enc_attn_scheduler, dec_attn_scheduler)

Starting epoch 1/30, enc lr scheduler: [0.001], dec lr scheduler: [0.001]
(Epoch 0, iter 50/787) Average loss so far: 6.972
(Epoch 0, iter 100/787) Average loss so far: 6.023
(Epoch 0, iter 150/787) Average loss so far: 5.968
(Epoch 0, iter 200/787) Average loss so far: 5.927
(Epoch 0, iter 250/787) Average loss so far: 5.877
(Epoch 0, iter 300/787) Average loss so far: 5.818
(Epoch 0, iter 350/787) Average loss so far: 5.785
(Epoch 0, iter 400/787) Average loss so far: 5.725
(Epoch 0, iter 450/787) Average loss so far: 5.683
(Epoch 0, iter 500/787) Average loss so far: 5.621
(Epoch 0, iter 550/787) Average loss so far: 5.551
(Epoch 0, iter 600/787) Average loss so far: 5.440
(Epoch 0, iter 650/787) Average loss so far: 5.360
(Epoch 0, iter 700/787) Average loss so far: 5.215
(Epoch 0, iter 750/787) Average loss so far: 5.093
Average epoch loss: 5.701
This epoch took 8.194433637460072 mins. Time remaining: 3.0 hrs 57.0 mins.


7it [00:01,  5.45it/s]


validation loss: 4.93167952128819


100%|██████████| 793/793 [00:08<00:00, 95.55it/s] 


BLEU score: 0.002800126985071343, METEOR score: 0.08576382563509898
Starting epoch 2/30, enc lr scheduler: [0.0009972883382072953], dec lr scheduler: [0.0009972883382072953]
(Epoch 1, iter 50/787) Average loss so far: 4.863
(Epoch 1, iter 100/787) Average loss so far: 4.706
(Epoch 1, iter 150/787) Average loss so far: 4.641
(Epoch 1, iter 200/787) Average loss so far: 4.523
(Epoch 1, iter 250/787) Average loss so far: 4.437
(Epoch 1, iter 300/787) Average loss so far: 4.366
(Epoch 1, iter 350/787) Average loss so far: 4.295
(Epoch 1, iter 400/787) Average loss so far: 4.248
(Epoch 1, iter 450/787) Average loss so far: 4.186
(Epoch 1, iter 500/787) Average loss so far: 4.133
(Epoch 1, iter 550/787) Average loss so far: 4.082
(Epoch 1, iter 600/787) Average loss so far: 4.041
(Epoch 1, iter 650/787) Average loss so far: 4.004
(Epoch 1, iter 700/787) Average loss so far: 3.956
(Epoch 1, iter 750/787) Average loss so far: 3.932
Average epoch loss: 4.277
This epoch took 8.073306282361349 mi

...

BLEU score: 0.05513984026373013, METEOR score: 0.2094436932678814
Starting epoch 30/30, enc lr scheduler: [1.2711661792704668e-05], dec lr scheduler: [1.2711661792704668e-05]
(Epoch 29, iter 50/787) Average loss so far: 2.732
(Epoch 29, iter 100/787) Average loss so far: 2.722
(Epoch 29, iter 150/787) Average loss so far: 2.715
(Epoch 29, iter 200/787) Average loss so far: 2.739
(Epoch 29, iter 250/787) Average loss so far: 2.727
(Epoch 29, iter 300/787) Average loss so far: 2.729
(Epoch 29, iter 350/787) Average loss so far: 2.730
(Epoch 29, iter 400/787) Average loss so far: 2.714
(Epoch 29, iter 450/787) Average loss so far: 2.725
(Epoch 29, iter 500/787) Average loss so far: 2.711
(Epoch 29, iter 550/787) Average loss so far: 2.733
(Epoch 29, iter 600/787) Average loss so far: 2.737
(Epoch 29, iter 650/787) Average loss so far: 2.723
(Epoch 29, iter 700/787) Average loss so far: 2.724
(Epoch 29, iter 750/787) Average loss so far: 2.724
Average epoch loss: 2.725
This epoch took 7.99

7it [00:01,  5.51it/s]


validation loss: 3.1326404299054826


100%|██████████| 793/793 [00:01<00:00, 595.70it/s]

BLEU score: 0.055078340546709854, METEOR score: 0.20826842611097787


In [14]:
save_model(encoder_attn, decoder_attn, f"{identifier}_last")

## Encoder-Decoder (Extension: pretrained embeddings)

Please download [pretrained embeddings](https://huggingface.co/stanfordnlp/glove/resolve/main/glove.840B.300d.zip) and extract the contents to the project folder.

In [14]:
reset_rng()

In [13]:
pretrained_embedding_dict = create_pretrained_embedding_dict("./glove.840B.300d.txt")

In [14]:
embedding_size=300
encoder_pretrained_embed = EncoderRNN(
    input_size=vocab.n_unique_words, embedding_size=embedding_size, hidden_size=HIDDEN_SIZE, 
    padding_value=vocab.word2index(PAD_WORD), pretrained_embedding_dict=pretrained_embedding_dict, 
    vocab=vocab).to(DEVICE)
# in the training script, decoder is always fed a non-end token and thus never needs to generate padding
# also it should never generate "<UNKNOWN>"
decoder_pretrained_embed = AttnDecoderRNN(
    embedding_size=embedding_size,hidden_size=HIDDEN_SIZE, output_size=vocab.n_unique_words-1,
    padding_val=vocab.word2index(PAD_WORD), dropout=DROPOUT, global_max_ing_len=MAX_INGR_LEN,
    pretrained_embedding_dict=pretrained_embedding_dict, vocab=vocab).to(DEVICE)

100%|██████████| 44315/44315 [00:00<00:00, 371565.78it/s]


29462/44315 (0.665%) words have pretrained embeddings


100%|██████████| 44314/44314 [00:00<00:00, 357392.06it/s]


29462/44314 (0.665%) words have pretrained embeddings


In [18]:
initial_lr=1e-3
min_lr = 1e-5
n_epochs = 30
batch_size=128
encoder_pretrained_embed_optimizer = optim.Adam(encoder_pretrained_embed.parameters(), lr=initial_lr)
decoder_pretrained_embed_optimizer = optim.Adam(decoder_pretrained_embed.parameters(), lr=initial_lr)
# enc_scheduler = CosineAnnealingLR(encoder_pretrained_embed_optimizer, T_max=n_epochs, eta_min=min_lr)
# dec_scheduler = CosineAnnealingLR(decoder_pretrained_embed_optimizer, T_max=n_epochs, eta_min=min_lr)
enc_pretrained_embed_scheduler = CosineAnnealingLR(encoder_pretrained_embed_optimizer, T_max=n_epochs, eta_min=min_lr)
dec_pretrained_embed_scheduler = CosineAnnealingLR(decoder_pretrained_embed_optimizer, T_max=n_epochs, eta_min=min_lr)
identifier="pretrained_emb_attn_adam_without_intermediate_tags_wd0_lr1e-3"

pre_epoch_losses, pre_val_epoch_losses, pre_log = train(
    encoder_pretrained_embed, decoder_pretrained_embed, encoder_pretrained_embed_optimizer, decoder_pretrained_embed_optimizer, 
    train_ds, n_epochs=n_epochs, vocab=vocab, decoder_mode="attention", batch_size=batch_size, 
    enc_lr_scheduler=enc_pretrained_embed_scheduler, dec_lr_scheduler=dec_pretrained_embed_scheduler, 
    dev_ds_val_loss = dev_ds_val_loss, dev_ds_val_met=dev_ds_val_met, identifier=identifier,
    verbose_iter_interval=50)

save_log(identifier, pre_log, encoder_pretrained_embed_optimizer, decoder_pretrained_embed_optimizer, 
         enc_pretrained_embed_scheduler, dec_pretrained_embed_scheduler)

Starting epoch 1/30, enc lr scheduler: [0.001], dec lr scheduler: [0.001]
(Epoch 0, iter 50/787) Average loss so far: 6.968
(Epoch 0, iter 100/787) Average loss so far: 6.040
(Epoch 0, iter 150/787) Average loss so far: 5.995
(Epoch 0, iter 200/787) Average loss so far: 5.955
(Epoch 0, iter 250/787) Average loss so far: 5.910
(Epoch 0, iter 300/787) Average loss so far: 5.859
(Epoch 0, iter 350/787) Average loss so far: 5.839
(Epoch 0, iter 400/787) Average loss so far: 5.759
(Epoch 0, iter 450/787) Average loss so far: 5.627
(Epoch 0, iter 500/787) Average loss so far: 5.404
(Epoch 0, iter 550/787) Average loss so far: 5.209
(Epoch 0, iter 600/787) Average loss so far: 5.001
(Epoch 0, iter 650/787) Average loss so far: 4.851
(Epoch 0, iter 700/787) Average loss so far: 4.692
(Epoch 0, iter 750/787) Average loss so far: 4.582
Average epoch loss: 5.528
This epoch took 8.08877116839091 mins. Time remaining: 3.0 hrs 54.0 mins.


7it [00:01,  5.60it/s]


validation loss: 4.4685172353472025


100%|██████████| 793/793 [00:09<00:00, 87.13it/s] 


BLEU score: 0.0031624657072353694, METEOR score: 0.0796926100408187
Starting epoch 2/30, enc lr scheduler: [0.0009972883382072953], dec lr scheduler: [0.0009972883382072953]
(Epoch 1, iter 50/787) Average loss so far: 4.410
(Epoch 1, iter 100/787) Average loss so far: 4.295
(Epoch 1, iter 150/787) Average loss so far: 4.269
(Epoch 1, iter 200/787) Average loss so far: 4.187
(Epoch 1, iter 250/787) Average loss so far: 4.128
(Epoch 1, iter 300/787) Average loss so far: 4.080
(Epoch 1, iter 350/787) Average loss so far: 4.027
(Epoch 1, iter 400/787) Average loss so far: 3.997
(Epoch 1, iter 450/787) Average loss so far: 3.949
(Epoch 1, iter 500/787) Average loss so far: 3.910
(Epoch 1, iter 550/787) Average loss so far: 3.871
(Epoch 1, iter 600/787) Average loss so far: 3.844
(Epoch 1, iter 650/787) Average loss so far: 3.814
(Epoch 1, iter 700/787) Average loss so far: 3.778
(Epoch 1, iter 750/787) Average loss so far: 3.759
Average epoch loss: 4.009
This epoch took 8.075345408916473 mi

...

BLEU score: 0.054989968499987295, METEOR score: 0.21584912823420402
Starting epoch 30/30, enc lr scheduler: [1.2711661792704668e-05], dec lr scheduler: [1.2711661792704668e-05]
(Epoch 29, iter 50/787) Average loss so far: 2.667
(Epoch 29, iter 100/787) Average loss so far: 2.655
(Epoch 29, iter 150/787) Average loss so far: 2.648
(Epoch 29, iter 200/787) Average loss so far: 2.672
(Epoch 29, iter 250/787) Average loss so far: 2.658
(Epoch 29, iter 300/787) Average loss so far: 2.662
(Epoch 29, iter 350/787) Average loss so far: 2.664
(Epoch 29, iter 400/787) Average loss so far: 2.649
(Epoch 29, iter 450/787) Average loss so far: 2.657
(Epoch 29, iter 500/787) Average loss so far: 2.646
(Epoch 29, iter 550/787) Average loss so far: 2.667
(Epoch 29, iter 600/787) Average loss so far: 2.670
(Epoch 29, iter 650/787) Average loss so far: 2.655
(Epoch 29, iter 700/787) Average loss so far: 2.656
(Epoch 29, iter 750/787) Average loss so far: 2.661
Average epoch loss: 2.658
This epoch took 7.

7it [00:01,  6.06it/s]


validation loss: 3.0425445352281844


100%|██████████| 793/793 [00:01<00:00, 619.58it/s]


BLEU score: 0.057802547829115924, METEOR score: 0.21431412239195574


## Extension 4: Multi-Layer Encoder-Decoder + Neurologic Decoding

In [24]:
reset_rng()

In [15]:
embedding_size=300
num_layers=3
encoder_multilayer_attn = EncoderRNN(vocab.n_unique_words, embedding_size=embedding_size, hidden_size=HIDDEN_SIZE, 
                          padding_value=vocab.word2index(PAD_WORD), num_lstm_layers=num_layers).to(DEVICE)
# in the training script, decoder is always fed a non-end token and thus never needs to generate padding
# also it should never generate "<UNKNOWN>"
# decoder = DecoderRNN(embedding_size=embedding_size,hidden_size=HIDDEN_SIZE, output_size=vocab.n_unique_words-2).to(DEVICE)
decoder_multilayer_attn = AttnDecoderRNN(embedding_size, hidden_size=HIDDEN_SIZE, output_size=vocab.n_unique_words-1, 
                              padding_val=vocab.word2index(PAD_WORD), dropout=DROPOUT, num_lstm_layers=num_layers).to(DEVICE)

In [26]:
initial_lr=1e-3
min_lr = 1e-5
n_epochs = 30
batch_size=128
encoder_multilayer_attn_optimizer = optim.Adam(encoder_multilayer_attn.parameters(), lr=initial_lr)
decoder_multilayer_attn_optimizer = optim.Adam(decoder_multilayer_attn.parameters(), lr=initial_lr)
enc_multilayer_attn_scheduler = CosineAnnealingLR(encoder_multilayer_attn_optimizer, T_max=n_epochs, eta_min=min_lr)
dec_multilayer_attn_scheduler = CosineAnnealingLR(decoder_multilayer_attn_optimizer, T_max=n_epochs, eta_min=min_lr)
identifier="multilayer_attn_adam_without_intermediate_tags_wd0_lr1e-3"

multilayer_attn_epoch_losses, multilayer_attn_val_epoch_losses, multilayer_attn_log = train(
    encoder_multilayer_attn, decoder_multilayer_attn, encoder_multilayer_attn_optimizer, decoder_multilayer_attn_optimizer, train_ds, 
    n_epochs=n_epochs, vocab=vocab, decoder_mode="attention", batch_size=batch_size, 
    enc_lr_scheduler=enc_multilayer_attn_scheduler, dec_lr_scheduler=dec_multilayer_attn_scheduler, 
    dev_ds_val_loss = dev_ds_val_loss, dev_ds_val_met=dev_ds_val_met, identifier=identifier,
    verbose_iter_interval=50)

save_log(identifier, multilayer_attn_log, encoder_multilayer_attn_optimizer, decoder_multilayer_attn_optimizer, 
         enc_multilayer_attn_scheduler, dec_multilayer_attn_scheduler)

Starting epoch 1/30, enc lr scheduler: [0.001], dec lr scheduler: [0.001]
(Epoch 0, iter 50/787) Average loss so far: 6.988
(Epoch 0, iter 100/787) Average loss so far: 6.058
(Epoch 0, iter 150/787) Average loss so far: 6.035
(Epoch 0, iter 200/787) Average loss so far: 6.032
(Epoch 0, iter 250/787) Average loss so far: 6.019
(Epoch 0, iter 300/787) Average loss so far: 6.011
(Epoch 0, iter 350/787) Average loss so far: 6.015
(Epoch 0, iter 400/787) Average loss so far: 6.006
(Epoch 0, iter 450/787) Average loss so far: 6.002
(Epoch 0, iter 500/787) Average loss so far: 5.990
(Epoch 0, iter 550/787) Average loss so far: 5.972
(Epoch 0, iter 600/787) Average loss so far: 5.961
(Epoch 0, iter 650/787) Average loss so far: 5.934
(Epoch 0, iter 700/787) Average loss so far: 5.930
(Epoch 0, iter 750/787) Average loss so far: 5.931
Average epoch loss: 6.053
This epoch took 8.784394307931263 mins. Time remaining: 4.0 hrs 14.0 mins.


7it [00:01,  5.15it/s]


validation loss: 5.933405944279262


100%|██████████| 7/7 [00:08<00:00,  1.15s/it]
/home/junn/miniconda3/envs/nlp/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
100%|██████████| 793/793 [00:06<00:00, 114.02it/s]


BLEU score: 5.8211892454135815e-80, METEOR score: 0.034986121334209475
Starting epoch 2/30, enc lr scheduler: [0.0009972883382072953], dec lr scheduler: [0.0009972883382072953]
(Epoch 1, iter 50/787) Average loss so far: 5.916
(Epoch 1, iter 100/787) Average loss so far: 5.902
(Epoch 1, iter 150/787) Average loss so far: 5.896
(Epoch 1, iter 200/787) Average loss so far: 5.880
(Epoch 1, iter 250/787) Average loss so far: 5.882
(Epoch 1, iter 300/787) Average loss so far: 5.891
(Epoch 1, iter 350/787) Average loss so far: 5.881
(Epoch 1, iter 400/787) Average loss so far: 5.874
(Epoch 1, iter 450/787) Average loss so far: 5.868
(Epoch 1, iter 500/787) Average loss so far: 5.830
(Epoch 1, iter 550/787) Average loss so far: 5.730
(Epoch 1, iter 600/787) Average loss so far: 5.635
(Epoch 1, iter 650/787) Average loss so far: 5.547
(Epoch 1, iter 700/787) Average loss so far: 5.400
(Epoch 1, iter 750/787) Average loss so far: 5.205
Average epoch loss: 5.724
This epoch took 8.680376549561819

...

BLEU score: 0.053924556203878635, METEOR score: 0.2136804902885653
Starting epoch 30/30, enc lr scheduler: [1.2711661792704668e-05], dec lr scheduler: [1.2711661792704668e-05]
(Epoch 29, iter 50/787) Average loss so far: 2.496
(Epoch 29, iter 100/787) Average loss so far: 2.494
(Epoch 29, iter 150/787) Average loss so far: 2.521
(Epoch 29, iter 200/787) Average loss so far: 2.511
(Epoch 29, iter 250/787) Average loss so far: 2.510
(Epoch 29, iter 300/787) Average loss so far: 2.511
(Epoch 29, iter 350/787) Average loss so far: 2.505
(Epoch 29, iter 400/787) Average loss so far: 2.493
(Epoch 29, iter 450/787) Average loss so far: 2.497
(Epoch 29, iter 500/787) Average loss so far: 2.521
(Epoch 29, iter 550/787) Average loss so far: 2.491
(Epoch 29, iter 600/787) Average loss so far: 2.503
(Epoch 29, iter 650/787) Average loss so far: 2.506
(Epoch 29, iter 700/787) Average loss so far: 2.509
(Epoch 29, iter 750/787) Average loss so far: 2.515
Average epoch loss: 2.506
This epoch took 8.6

7it [00:01,  5.22it/s]


validation loss: 3.0682195595332553


100%|██████████| 793/793 [00:01<00:00, 777.01it/s]

BLEU score: 0.05360497700665978, METEOR score: 0.21295206921320584


---

## Evaluation

In [16]:
all_ingredients_lst = get_all_ingredients("./ingredient_set.json")

### Without attention

In [16]:
load_model(encoder, decoder, "adam_without_intermediate_tags_with_val_wd0_lr1e-3_ep_24")

In [17]:
all_decoder_outs, all_gt_recipes, all_gt_ingredients = eval(
    encoder, decoder, test_ds, vocab, batch_size=128, decoder_mode="basic",
    max_recipe_len=MAX_RECIPE_LEN)

  0%|          | 0/7 [00:00<?, ?it/s]

100%|██████████| 7/7 [00:05<00:00,  1.30it/s]


In [18]:
calc_bleu(all_gt_recipes, all_decoder_outs)

0.04449619754164479

In [19]:
calc_meteor(all_gt_recipes, all_decoder_outs, split_gt=False)

  0%|          | 0/774 [00:00<?, ?it/s]

100%|██████████| 774/774 [00:03<00:00, 209.73it/s]


0.20156387133765763

In [21]:
convert_eval_out_to_get_ingredient_metrics(all_decoder_outs, all_gt_ingredients,
                                           vocab, all_ingredients_lst)

774it [00:00, 1141.23it/s]

Avg. % given ingredients: 25.563%
Avg. number of extra ingredients: 2.978


### With attention

In [45]:
load_model(encoder_attn, decoder_attn, "attn_adam_without_intermediate_tags_wd0_lr1e-3_ep_26")

In [46]:
all_decoder_outs, all_gt_recipes, all_gt_ingredients = eval(
    encoder_attn, decoder_attn, test_ds, vocab, batch_size=128, decoder_mode="attention",
    max_recipe_len=MAX_RECIPE_LEN)

  0%|          | 0/7 [00:00<?, ?it/s]

100%|██████████| 7/7 [00:06<00:00,  1.04it/s]


In [47]:
calc_bleu(all_gt_recipes, all_decoder_outs)

0.04975297680717382

In [48]:
calc_meteor(all_gt_recipes, all_decoder_outs, split_gt=False)

  0%|          | 0/774 [00:00<?, ?it/s]

100%|██████████| 774/774 [00:02<00:00, 292.46it/s]


0.20857380991725616

In [26]:
convert_eval_out_to_get_ingredient_metrics(all_decoder_outs, all_gt_ingredients,
                                           vocab, all_ingredients_lst)

774it [00:00, 1107.21it/s]

Avg. % given ingredients: 33.453%
Avg. number of extra ingredients: 2.295


## Attention + pretrained embeddings

In [49]:
load_model(encoder_pretrained_embed, decoder_pretrained_embed, 
           "pretrained_emb_attn_adam_without_intermediate_tags_wd0_lr1e-3_ep_29")

In [56]:
all_decoder_outs, all_gt_recipes, all_gt_ingredients = eval(
    encoder_pretrained_embed, decoder_pretrained_embed, test_ds, vocab, batch_size=128, 
    decoder_mode="attention", max_recipe_len=MAX_RECIPE_LEN)

  0%|          | 0/7 [00:00<?, ?it/s]

100%|██████████| 7/7 [00:07<00:00,  1.04s/it]


In [57]:
calc_bleu(all_gt_recipes, all_decoder_outs)

0.05573056259570414

In [58]:
calc_meteor(all_gt_recipes, all_decoder_outs, split_gt=False)

100%|██████████| 774/774 [00:02<00:00, 289.11it/s]


0.20985559115168426

In [36]:
convert_eval_out_to_get_ingredient_metrics(all_decoder_outs, all_gt_ingredients,
                                           vocab, all_ingredients_lst)

774it [00:00, 1054.15it/s]

Avg. % given ingredients: 31.074%
Avg. number of extra ingredients: 2.104


## Attention + Multilayer + Neurologic Decoding

### Without Neurologic Decoding

In [22]:
load_model(encoder_multilayer_attn, decoder_multilayer_attn, "multilayer_attn_adam_without_intermediate_tags_wd0_lr1e-3_ep_14")

FileNotFoundError: [Errno 2] No such file or directory: 'saved_models/encoder_multilayer_attn_adam_without_intermediate_tags_wd0_lr1e-3_ep_14.pth'

In [38]:
all_decoder_outs_multi, all_gt_recipes_multi, all_gt_ingredients_multi = eval(
    encoder_multilayer_attn, decoder_multilayer_attn, test_ds, vocab, batch_size=128, decoder_mode="attention",
    max_recipe_len=MAX_RECIPE_LEN)

100%|██████████| 7/7 [00:02<00:00,  2.64it/s]


In [39]:
calc_bleu(all_decoder_outs_multi, all_gt_recipes_multi)

0.053654397203939004

In [40]:
calc_meteor(all_gt_recipes_multi, all_decoder_outs_multi, split_gt=False)

100%|██████████| 774/774 [00:01<00:00, 724.46it/s]


0.21415571426435778

In [41]:
convert_eval_out_to_get_ingredient_metrics(all_decoder_outs_multi, all_gt_ingredients_multi,
                                           vocab, all_ingredients_lst)

774it [00:00, 1134.83it/s]

Avg. % given ingredients: 32.267%
Avg. number of extra ingredients: 2.223


### With Neurologic Decoding

! WARNING: this code takes around 20-30 mins to run

In [46]:
k=3
alpha=50 # likelihood
beta = 2 # num constraints satisfied
neg_constraint_penalty, likelihood_penalty, low_irr_satisfaction_penalty = 10, 0.5, 0.5
lam = 5.0

all_decoder_outs, all_gt_recipes, all_gt_ings = eval_neuro_decoding(
    encoder_multilayer_attn, decoder_multilayer_attn, test_ds, vocab, all_ingredients_lst, k=k, alpha=alpha, beta=beta, 
    neg_constraint_penalty=neg_constraint_penalty, likelihood_penalty=likelihood_penalty, 
    low_irr_satisfaction_penalty=low_irr_satisfaction_penalty, lam=lam)


100%|██████████| 774/774 [16:36<00:00,  1.29s/it]


In [47]:
calc_bleu(all_gt_recipes, all_decoder_outs)

0.05810241560787664

In [48]:
calc_meteor(all_gt_recipes, all_decoder_outs, split_gt=False)

100%|██████████| 774/774 [00:01<00:00, 712.33it/s]


0.23339527798724602

In [49]:
convert_eval_out_to_get_ingredient_metrics(all_decoder_outs, all_gt_ings,
                                           vocab, all_ingredients_lst, skip_ing_processing=True)

774it [00:00, 1072.67it/s]

Avg. % given ingredients: 79.462%
Avg. number of extra ingredients: 0.274


---

## Metric Sample

In [51]:
all_ings = get_all_ingredients("./ingredient_set.json")
all_ings_regex = get_ingredients_regex(all_ings)
metric_sample_ings, metric_sample_gold_recipe, metric_sample_generated_recipe = \
    load_metric_sample("./metric_sample.txt")

In [52]:
prop_inp_ings, n_extra_ings = get_prop_input_num_extra_ingredients_m(
    metric_sample_ings, metric_sample_generated_recipe, all_ings_regex, verbose=True)
print(f"\nproportion of input ingredients: {prop_inp_ings}\nnumber of extra ingredients: {n_extra_ings}")

=====Input ingredients in text=====
['lemon juice', 'orange juice', 'water', 'sugar', 'strawberries']

=====All ingredients in text===== 
['lemon juice', 'orange juice', 'water', 'cantaloupe', 'vanilla ice cream', 'sugar', 'strawberries']

proportion of input ingredients: 1.0
number of extra ingredients: 2


In [55]:
bleu_score = calc_bleu([metric_sample_gold_recipe], [metric_sample_generated_recipe], split_gt=True, split_gen=True)
meteor_score = calc_meteor([metric_sample_gold_recipe], [metric_sample_generated_recipe], split_gt=True, split_gen=True)
print(f"BLEU score: {bleu_score}, METEOR score: {meteor_score}")

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00, 458.90it/s]

BLEU score: 0.2757364975156813, METEOR score: 0.5479209577754892


---

# Qualitative Evaluation

In [29]:
ingredients_sample_orig = "2 c sugar, 1/4 c lemon juice, 1 c water, 1/3 c orange juice, 8 c strawberries"

perform relevant preprocessing steps (note: these operations are taken from `preprocess_data` and is the same preprocessing done on the dataset)

In [ ]:
def prepare_qual_ingredients()

In [30]:
ingredients_sample = re.sub("([^0-9a-zA-Z.'\"/ ])", r" \1 ", ingredients_sample_orig)
ingredients_sample = '<INGREDIENT_START> ' + ingredients_sample + ' <INGREDIENT_END>'
ingredients_sample = re.sub('[ ]{2,}', " ", ingredients_sample)

In [32]:
ingredients_sample

'<INGREDIENT_START> 2 c sugar , 1/4 c lemon juice , 1 c water , 1/3 c orange juice , 8 c strawberries <INGREDIENT_END>'

In [37]:
ingredients_sample_idxs = torch.tensor([vocab.word2index(w) for w in ingredients_sample.split(" ")],
                                       dtype=torch.long, device=DEVICE)
# convert to batch form with batch size 1 as model is expecting batched input
ingredients_sample_idxs = ingredients_sample_idxs[None] 

In [38]:
ingredients_sample_idxs

tensor([[   0,   22,   12,   16,   78,   32,   12,  641,  900,   78,   17,   12,
          311,   78,  235,   12,  791,  900,   78,  385,   12, 1218,    1]],
       device='cuda:0')

In [39]:
ingr_lens = torch.tensor([len(x) for x in ingredients_sample_idxs], dtype=torch.long, device=DEVICE)

In [40]:
ingr_lens

tensor([23], device='cuda:0')

In [41]:
qual_res_basic = get_predictions_iter(ingredients_sample_idxs, ingr_lens, encoder, decoder,
                                        vocab)

In [43]:
' '.join(qual_res_basic[0])

'<RECIPE_START> combine sugar , cornstarch and water in a saucepan bring to a boil , stirring constantly boil for 1 minute remove from heat and stir in lemon juice and lemon juice pour into a large bowl and stir in the orange juice and lemon juice pour into a large bowl and stir in the orange juice and lemon juice pour into a large bowl and chill <RECIPE_END>'

In [60]:
qual_res_attn = get_predictions_iter(ingredients_sample_idxs, ingr_lens, encoder_attn, decoder_attn,
                                    vocab, decoder_mode="attention")

In [62]:
' '.join(qual_res_attn[0])

'<RECIPE_START> combine sugar , water , lemon juice and salt in a saucepan bring to a boil , stirring constantly boil for 5 minutes , stirring constantly remove from heat and stir in lemon juice and pour into a sterilized jars seal and store in refrigerator <RECIPE_END>'

In [63]:
qual_res_attn_pre_embed = get_predictions_iter(
    ingredients_sample_idxs, ingr_lens, encoder_pretrained_embed, decoder_pretrained_embed,
    vocab, decoder_mode="attention")

In [65]:
' '.join(qual_res_attn_pre_embed[0])

'<RECIPE_START> combine sugar , water , and lemon juice in a saucepan bring to a boil over medium heat , stirring constantly , until sugar dissolves remove from heat and cool to room temperature add vanilla and mix well chill <RECIPE_END>'